In [97]:
"""
Here the particles are made to converge so as to find a sequence
which is able to achieve the target score
"""
import random
import sys
import numpy as np
import copy

In [98]:
TARGET = 50
MAX_INPUTS = 10
MAX_PARTICLES = 10
V_MAX = 10 #Maximum velocity change allowed
MAX_EPOCHS = 200
START_RANGE_MIN = 140
START_RANGE_MAX = 200

In [99]:
class Particle:
    def __init__(self, pBest):
        self.data = [0] *  MAX_INPUTS
        self.pBest = pBest
        self.velocity = 0.0
    
    def get_data(self):
        return self.data
    
    def set_data(self, value):
        self.data = value
    
    def set_pBest(self, value):
        self.pBest = value
        
    def get_pBest(self):
        return self.pBest
    
    def get_velocity(self):
        return self.velocity
    
    def set_velocity(self, value):
        self.velocity = value

In [100]:
def initialize_Particles():
    particles = []
    for i in range(MAX_PARTICLES):
        newParticle = Particle(0)
        total = 0
        data = []
        for j in range(MAX_INPUTS):
            data.append(np.floor(np.random.uniform(START_RANGE_MIN, START_RANGE_MAX)))
        
        newParticle.set_data(data)
        newParticle.set_pBest(np.sum(data))
        particles.append(copy.deepcopy(newParticle))
    
    return particles

In [101]:
def test_problem(index, particles):
    val = 0
    a = particles[index].get_data()
    for i in a:
        val += i
    return val

In [102]:
def max_fitness(particles):
    minimum = 0
    for i in range(MAX_PARTICLES):
        #print("i: " + str(i))
        if i!=minimum: 
            if np.abs(TARGET - test_problem(i, particles))<np.abs(TARGET - test_problem(minimum, particles)):
                minimum = i
    
    return minimum

In [103]:
def calc_velocity(gBestIndex, particles):
    
    bestVal = test_problem(gBestIndex, particles)
    for i in range(MAX_PARTICLES):
        testVal = test_problem(i, particles)
        velocity = particles[i].get_velocity() + (2 * np.random.uniform() * (bestVal - testVal))
        velocity += (2 * np.random.uniform() * (particles[i].get_pBest() - testVal))
        
        if velocity > V_MAX:
            velocity = V_MAX
        elif velocity < -V_MAX:
            velocity = -V_MAX
        
        particles[i].set_velocity(velocity)

In [104]:
def update_particles(particles, gBest):
    for i in range(MAX_PARTICLES):
        data = particles[i].get_data()
        temp = particles[gBest].get_data()
        vel = particles[i].get_velocity()
        for y in range(len(data)):
            if data[y] != temp[y]:
                data[y] += vel
                data[y]=np.floor(data[y])
        particles[i].set_data(data)
        
        total = test_problem(i, particles)
        if np.abs(TARGET - total) < particles[i].get_pBest():
            particles[i].set_pBest(total)

In [105]:
def PSO_algo():
    epoch = 1
    result = []
    solution = -1
    
    particles = initialize_Particles()
    random.seed(10)
    
    while epoch<MAX_EPOCHS:
        for i in range(MAX_PARTICLES):
            if test_problem(i, particles)==TARGET:
                result = particles[i].get_data()
                solution = i
                break
        
        if solution>=0:
            print(result)
            break
        
        gBest = max_fitness(particles)
        
        calc_velocity(gBest, particles)
        update_particles(particles, gBest)
        
        #print("Best in epoch " + str(epoch) +": " + str(particles[gBest].get_data()))
        print(str(np.sum(particles[gBest].get_data())))
        epoch += 1        

In [111]:
#print(max_fitness())
PSO_algo()

1572.0
1486.0
1419.0
1333.0
1256.0
1168.0
1072.0
1002.0
904.0
806.0
739.0
640.0
543.0
457.0
378.0
312.0
216.0
124.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
49.0
[8.0, 26.0, 10.0, 0.0, 16.0, -17.0, -2.0, 32.0, -23.0, 0.0]
